In [ ]:
import pandas as pd
from datetime import datetime
import plotly.express as px
import time
import chart_studio.plotly as py

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.path as path
import matplotlib.ticker as ticker
import matplotlib.animation as animation
from IPython.display import HTML
from matplotlib import cm
import IPython.display as display
from mpl_toolkits.axes_grid1 import make_axes_locatable

In [ ]:
title_basic = pd.read_csv('title.basics.tsv', sep='\t')
title_episode = pd.read_csv('title.episode.tsv', sep='\t')
title_ratings = pd.read_csv('title.ratings.tsv', sep='\t')

In [ ]:
title_basic_modified = title_basic
basic_ratings_df = pd.merge(title_basic_modified,
                            title_ratings, 
                            on='tconst',
                            how='inner')

title_overall_df = pd.merge(basic_ratings_df,
                            title_episode,
                            on='tconst',
                            how='left')

print(title_overall_df)

In [ ]:
final_df_with_top10 = pd.DataFrame()
for i in range(1990,2021):
    df_with_top30 = title_overall_df[(title_overall_df['startYear'] == str(i))]\
        .groupby(['parentTconst', 'startYear'])\
        .agg({'averageRating': 'mean',
                  'numVotes':'mean',
                  'startYear':'count'})\
        .sort_values(by=['numVotes','averageRating'],
                     ascending=False).head(30)
    
    df_with_top10 = df_with_top30.sort_values(by='averageRating',
                                              ascending=False).head(10)
    df_with_top10.rename(columns={'startYear': 'SeasonEpisodeCount'},inplace=True)
    df_with_top10.reset_index(inplace=True)    
    def name_of_series(x):
        return list(title_overall_df[title_overall_df['tconst'] == x]['primaryTitle'])[0]
        
    df_with_top10['name_of_series'] = df_with_top10['parentTconst']\
        .map(lambda x: name_of_series(x))
    
    final_df_with_top10 = final_df_with_top10.append(df_with_top10, ignore_index=True)
            
print(final_df_with_top10)

In [ ]:
final_df_with_top10.rename(columns={'startYear': 'Year', 'numVotes': 'Votes'}, inplace=True)

In [ ]:
final_df_with_top1990 = final_df_with_top10[final_df_with_top10['Year'] == '1995']

In [ ]:
fig, ax = plt.subplots(figsize=(15, 8))
list_of_votes = list(final_df_with_top1990['Votes'])
max_vote = max(list_of_votes)
div = make_axes_locatable(ax)
cax = div.append_axes('right', '5%', '5%')
plot = plt.scatter(list_of_votes, list_of_votes, c = list_of_votes, cmap = 'Pastel1')
cbar = plt.colorbar(plot,cax=cax)
cbar.ax.get_yaxis().labelpad = 15
cbar.ax.text(-0.6, 1.02,'Average Votes', transform=cbar.ax.transAxes, weight=7, size=12)

def draw_barchart(year):
    dff = final_df_with_top10[final_df_with_top10['Year'].eq(str(year))].sort_values(by='averageRating', ascending=True).tail(10)
    list_of_votes = list(dff['Votes'])
    max_vote = max(list_of_votes)
    ax.clear()
    ax.barh(dff['name_of_series'], 
            dff['averageRating'], 
            color=cm.Pastel1( [i/ max_vote for i in list_of_votes] ))
    

    for i, (value, name) in enumerate(zip(dff['averageRating'], dff['name_of_series'])):
        ax.text(value + -0.05, i,     name,           size=14, weight=600, ha='right', va='center')
        ax.text(value + 0.05, i,     f'{round(value,2)}',  size=14, ha='left',  va='center')

    ax.text(1, 0.2, year, transform=ax.transAxes, color='#777777', size=46, ha='right', weight=800)
    ax.text(0, 1.06, 'IMDB Average Ratings', transform=ax.transAxes, size=15,weight=600, color='#777777')
    ax.xaxis.set_major_formatter(ticker.StrMethodFormatter('{x:,.0f}'))
    ax.xaxis.set_ticks_position('top')
    ax.set_xlim([0,13])
    ax.tick_params(axis='x', colors='black', labelsize=12)
    ax.set_yticks([])
    ax.margins(0, 0.01)

    ax.set_axisbelow(True)
    ax.text(0.1, 1.1, 'IMDB Rating of Top 10 Famous Series from 1991 to 2020',
            transform=ax.transAxes, size=25, weight=800, ha='left')
    ax.text(0, -0.05, 'Source : https://datasets.imdbws.com/', transform=ax.transAxes, size=15,weight=600, color='#777777')
    ax.spines['right'].set_visible(False)
    ax.spines['bottom'].set_visible(False)
    plt.box(False)
    
    
draw_barchart(2010)

In [ ]:

from IPython.display import HTML
fig, ax = plt.subplots(figsize=(15, 8),dpi=300)
list_of_votes = list(final_df_with_top1990['Votes'])
max_vote = max(list_of_votes)
div = make_axes_locatable(ax)
cax = div.append_axes('right', '5%', '5%')
plot = plt.scatter(list_of_votes, list_of_votes, c = list_of_votes, cmap = 'Pastel1')
cbar = plt.colorbar(plot,cax=cax)
cbar.ax.get_yaxis().labelpad = 15
cbar.ax.text(-0.6, 1.02,'Average Votes', transform=cbar.ax.transAxes, weight=7, size=12)
animator = animation.FuncAnimation(fig, draw_barchart, frames=range(1991, 2021), interval=3000, repeat=True)
HTML(animator.to_html5_video()) 
HTML(animator.to_jshtml()) 

In [ ]:
Writer = animation.writers['ffmpeg']
writer = Writer(fps=1, bitrate=64000, codec='mpeg4')
animator.save('1991To2020.mp4', writer=writer)